In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.docstore.document import Document

llm = Ollama(model="llama3")

In [3]:

loader_temp = PyPDFLoader('/home/azureuser/cloudfiles/code/rag_boilerplate_api/output.pdf')
docs_temp = loader_temp.load_and_split()
for doc in docs_temp:
    doc.page_content = doc.page_content.replace('\n', ' ')


In [4]:
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-large-en-v1.5')
# Load PDFs from directory and prepare Chroma database if PDFs exist.

db = Chroma.from_documents(docs_temp, embeddings)


/anaconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/anaconda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/anaconda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [30]:
source_filter = '/home/azureuser/cloudfiles/code/rag_boilerplate_api/output.pdf'
filtered_output = db._collection.get()

filtered_ids = filtered_output['ids']
print(filtered_ids)

TypeError: Collection.get() got an unexpected keyword argument 'filter'

In [33]:
source_filter = '/home/azureuser/cloudfiles/code/rag_boilerplate_api/output.pdf'
filtered_output = db.get(where={'metadatas.source': source_filter})

filtered_ids = filtered_output['ids']
print(filtered_ids)

[]


In [41]:
 db.get()

{'ids': ['2970fcbf-cfa5-415e-94fe-9d3819ccea4e',
  '46ec2f6c-fb40-4bd6-be62-06231de43c96',
  '4733e389-655f-4d68-b12b-be1e4b67fdab',
  '4e39ec7e-a2d0-4064-a4f4-b23a187de2e8',
  '516dfa38-0dd1-4c17-beaa-66b56d6e7aaf',
  '58b56386-54ff-453b-baec-c19353d3d9cb',
  '63d99672-769f-48b5-a978-c78c9bcdf601',
  '93357dca-22c8-4fc7-868d-6cb607fbc89d',
  '94ffaf67-e518-49fc-b6cb-2b2864a0aa4c',
  'b725e67f-1f3a-4f41-a884-b8f622f0694d',
  'c1da45a3-f869-4464-b1f5-0119b127e849',
  'c1ea7b98-65e4-49ea-8ff5-39e61661bc2d',
  'cbcd0a11-d7a4-4b84-847c-b1928f42e06c',
  'dd38edb1-128e-4836-82e1-d8b61b5b2a47',
  'ebfe7828-0dda-401d-b2bb-a39499e536b5',
  'fbe53ea5-141a-404b-8c1b-85df856ab475'],
 'embeddings': None,
 'metadatas': [{'page': 2,
   'source': '/home/azureuser/cloudfiles/code/rag_boilerplate_api/output.pdf'},
  {'page': 0,
   'source': '/home/azureuser/cloudfiles/code/rag_boilerplate_api/output.pdf'},
  {'page': 1,
   'source': '/home/azureuser/cloudfiles/code/rag_boilerplate_api/output.pdf'},
  {'

In [47]:
def delete_pdf(doc_path, username):

    data = db.get()
    filtered_ids = [
        data['ids'][i]
        for i in range(len(data['metadatas']))
        if data['metadatas'][i]['source'] == doc_path and data['metadatas'][i]['user'] == username
    ]
    db.delete(filtered_ids)

    return f'Given document: {doc_path} for the user: {username} deleted from the db'

In [45]:
db.delete(filtered_ids)

In [46]:
db.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [48]:
import os

def upload_pdf_documents(filepath, username):
    """Load and add PDF documents to the Chroma database."""
    file_name = os.path.basename(filepath)
    if file_name.lower().endswith('.pdf'):
        loader_temp = PyPDFLoader(filepath)  # Assuming PyPDFLoader is defined elsewhere
        # Assuming text_splitter is defined elsewhere or passed as an argument if needed
        docs_temp = loader_temp.load_and_split(text_splitter=text_splitter)

        for doc in docs_temp:
            # Replace newline characters with spaces
            doc.page_content = doc.page_content.replace('\n', ' ')

            # Check if 'metadata' exists and is a dictionary
            if not hasattr(doc, 'metadata'):
                doc.metadata = {}
            if not isinstance(doc.metadata, dict):
                doc.metadata = {}

            # Add or update the 'user' field in the metadata
            doc.metadata['user'] = username

        # Assuming db is defined and has a method add_documents
        db.add_documents(docs_temp)
    else:
        print(f"Skipping non-PDF file: {file_name}")

In [49]:
db = Chroma.from_documents([Document(page_content="")], embeddings)

In [51]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=20, length_function=len, is_separator_regex=False)

In [52]:
upload_pdf_documents(filepath='/home/azureuser/cloudfiles/code/rag_boilerplate_api/output.pdf', username='ricardo')

In [1]:
import sqlite3

# Example of using a context manager to handle SQLite connections
with sqlite3.connect('/home/azureuser/cloudfiles/code/rag_boilerplate_api/info.sqlite') as conn:
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS test (id INTEGER PRIMARY KEY, name TEXT)")
    conn.commit()

OperationalError: database is locked